In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import scipy
import math
from fairtree import * 
from gmm_fairlet_decomposition import VanillaFairletDecomposition, MCFFairletDecomposition
from scipy.spatial import distance
from MWD_utils import *
from sklearn.mixture import GaussianMixture
%matplotlib inline

df = pd.read_csv('Data/income/adult.data',sep=',',header=None)
df[9].replace({" Male": 1, " Female": 0}, inplace=True)
A = df[9].to_numpy()
df = df[[9,0,2,4,10,12]]
data = df.to_numpy()
print('Dataset fraction',sum(A)/len(A))
np.random.seed(0) 
n_class0 = 170
n_class1 = 330
selection = np.random.choice(np.where(data[:,0]==0)[0],size=n_class0,replace=False)
data0 = data[selection]
selection = np.random.choice(np.where(data[:,0]==1)[0],size=n_class1,replace=False)
data1 = data[selection]
data = np.concatenate((data0,data1))
np.random.shuffle(data)
A = data[:,0] 
data = data[:,1:] 

# Scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data= scaler.fit_transform(data) 
print(data.shape)
reds = np.where(A==0)[0]
blues = np.where(A==1)[0]
print('fraction in subsample',sum(A)/len(A))

# Kmedian decomposition

In [ ]:
# Kmedian decomposition construction does not depend on a changing metric space for the different number of 
# mixture components, so we can generate it outside the experiment loop. 

# Generating (1,2)-fairlet decomposition
p = 1 
q = 2 

print("Constructing tree...")
fairlet_s = time.time()
root = build_quadtree(data)

print("Doing fair clustering...")
_, kmedian_fairlet_centers, kmedian_fairlets = tree_fairlet_decomposition(p, q, root, data, A)
fairlet_e = time.time()

print(f'N fairlets {len(kmedian_fairlet_centers)}')

# Running experiments

In [ ]:
# Control variables for experiments 
# Runtime is approximately 1 minute pr iteration. I.e. total_runtime ≈ 1min * iterations * (n_components-1)

n_components = 20      # The largest number of components. Will run experiments from 2 to n_components
iterations   = 5      # Number of iterations of different random seeds for each number of mixture components
cov_type     = 'full' # Covariance structure of the GMM. Options: {‘full’, ‘tied’, ‘diag’}

In [ ]:
# Initializing arrays to save results
# Full is the traditional / colorblind GMM model on the full original dataset.
# The traditional model does not use a decomposition, so we do not need to initialize it.
# mcf is the GMM MWD minimum cost flow decomposition. 
Cost_full    = np.zeros([19,2])
Cost_mcf     = np.zeros([19,2])
Cost_kmedian = np.zeros([19,2])

Decomp_mcf     = np.zeros([19,2])
Decomp_kmedian = np.zeros([19,2])

Balance_full    = np.zeros([19,2])
Balance_mcf     = np.zeros([19,2])
Balance_kmedian = np.zeros([19,2])

Entropy_full    = np.zeros([19,2])
Entropy_mcf     = np.zeros([19,2])
Entropy_kmedian = np.zeros([19,2])

Like_mcf     = np.zeros([19,2])
Like_kmedian = np.zeros([19,2])


k = data.shape[1] # multivariate dimension of the data.
for components in range(2,n_components+1):

    # Running lists for each number of mixture components for the different random seeds 
    # Balance (B), total cost (C), entropy (H), decomposition cost (D), and decomposition likelihood (L)
    
    B_full = []
    C_full = []
    H_full = []


    D_mcf = []
    B_mcf = []
    C_mcf= []
    H_mcf = []

    D_kmedian = []
    B_kmedian = []
    C_kmedian= []
    H_kmedian = []
    
    L_mcf = []
    L_kmedian = []

    print('\nComponents:',components)

    for random_state in range(iterations):
        print('\nIteration',random_state+1, 'of',iterations)
    
        # ======== Colorblind GMM ========
        gm_trad = GaussianMixture(n_components=components, random_state=random_state,covariance_type=cov_type,max_iter=100,n_init=3,init_params='k-means++').fit(data)
        gamma_metric = gm_trad.predict_proba(data)

        # Saving colorblind distribution parameters to instantiate G metric and compute model-weighted distances
        metric_weights = gm_trad.weights_
        metric_means = gm_trad.means_
        metric_covariances = gm_trad.covariances_

        cost_full = MHD_GMM_Cost(gamma_metric,data,metric_weights,metric_means,metric_covariances) 
        print('Traditional Cost',cost_full)        

        # Fairness for the colorblind model 
        H1,H_ratio1,balance1,p1 = soft_fairness(gamma_metric,A)   # for category 1 
        H2,H_ratio2,balance2,p2 = soft_fairness(gamma_metric,1-A) # for category 2
        balance = np.min([balance1,balance2])
        H_ratio = np.min([H_ratio1,H_ratio2])
     
        C_full.append(cost_full)
        B_full.append(balance)
        H_full.append(H_ratio)
        print('Full Balance',balance)
      
    
        # ======== GMM MWD MCF =========
        print('GMM MCF decomposition...')
        # Instantiating the MCF Decomposition. Running (1,2)-fairlet decomposition
        mcf = MCFFairletDecomposition(list(blues), list(reds), 2, None, list(data),metric_weights,metric_means,metric_covariances)

        # Computing the distance matrix between blue and red nodes
        mcf.compute_distances()

        # Adding nodes and edges
        mcf.build_graph(plot_graph=False)

        # Decomposing for fairlets 
        mcf_fairlets, mcf_fairlet_centers, mcf_fairlet_costs = mcf.decompose()
    
        # Finding the decomposition cost
        decomp_cost = []
        for f in mcf_fairlets:
            decomp_cost.append(center_cost(data[f],metric_weights,metric_means,metric_covariances))   
        decomp_cost = sum(decomp_cost)
        print('MCF decomposition cost:',decomp_cost)

        # Identifying fairlet centers  
        mcf_centers_mean = []
        for i in range(len(mcf_fairlets)):
            mcf_centers_mean.append(np.mean(data[mcf_fairlets[i]],axis=0))
        mcf_centers_mean = np.array(mcf_centers_mean)
        
        #Likelihood of decomposition
        mcf_fairlet_likelihood = []
        for i in range(len(mcf_fairlets)):
            for j in range(len(data[mcf_fairlets[i]])):  
                l = mcf_fairlets[i][j] # index of data point x
                prob_at_x = gm_trad.predict_proba(data[l:l+2])[0] # the component probability at x. l+2 to make 'predict_proba' work. Only saing the first entry [0]
                dist,G = MWDG(data[mcf_fairlet_centers[i]],data[mcf_fairlets[i]][j],metric_weights,metric_means,metric_covariances,prob_at_x)                
                mcf_fairlet_likelihood.append(logL(k,G,dist,G_metric=True)) 
                  
                    
        decomp_likelihood = np.sum(mcf_fairlet_likelihood)  # likelihood cost
       

        # Algorithm 1 on GMM MWD decomposition 
        fairlet_sizes = [len(mcf_fairlets[i]) for i in range(len(mcf_fairlet_centers))] # size of fairlets
        Dbar=[]
        for i in range(len(mcf_fairlet_centers)):
            for j in range(fairlet_sizes[i]):
                Dbar.append(mcf_centers_mean[i])
        Dbar = np.asarray(Dbar)
        
        # Computing mixture model on Dbar
        gm = GaussianMixture(n_components=components, random_state=random_state,covariance_type=cov_type,max_iter=100,n_init=3,init_params='k-means++').fit(Dbar)
        gamma = gm.predict_proba(Dbar)
        weights = gm.weights_
        means = gm.means_
        covariances = gm.covariances_

        Dbar_cost = MHD_GMM_Cost(gamma,Dbar,weights,means,covariances)
        total_cost = decomp_cost + Dbar_cost
        
        print('MCF Total cost:',total_cost)        
                
        # Assigning center responsibilities to fairlet members with the right index 
        center_gamma = gm.predict_proba(mcf_centers_mean) # responsibility of centers
        gamma=np.zeros([data.shape[0],components])
        for i in range(len(mcf_centers_mean)):
                gamma[mcf_fairlets[i]] = center_gamma[i]
    
        # Fairness
        H1,H_ratio1,balance1,p1 = soft_fairness(gamma,A)   # For category 1 
        H2,H_ratio2,balance2,p2 = soft_fairness(gamma,1-A) # For category 2
        balance = np.min([balance1,balance2])
        H_ratio = np.min([H_ratio1,H_ratio2])
    
        D_mcf.append(decomp_cost)
        C_mcf.append(total_cost)
        B_mcf.append(balance)
        H_mcf.append(H_ratio)
        L_mcf.append(decomp_likelihood)
    

        # ======== Euclidean Kmedian Decomp ======== 
        
        # Finding GMM cost of kmedian decomposition 
        kmedian_fairlet_costs = []
        for i in range(len(kmedian_fairlets)):
            for j in range(len(data[kmedian_fairlets[i]])):
                kmedian_fairlet_costs.append(MWDistance(data[kmedian_fairlet_centers[i]],data[kmedian_fairlets[i]][j],metric_weights,metric_means,metric_covariances))  
        decomp_cost = sum(kmedian_fairlet_costs)
        
        #Likelihood of decomposition
        kmedian_fairlet_likelihood = []
        for i in range(len(kmedian_fairlets)):
            for j in range(len(data[kmedian_fairlets[i]])):  
                l = kmedian_fairlets[i][j] # index of data point x
                prob_at_x = gm_trad.predict_proba(data[l:l+2])[0] # the component probability at x. l+2 to make 'predict_proba' work. Only saing the first entry [0]
                dist,G = MWDG(data[kmedian_fairlet_centers[i]],data[kmedian_fairlets[i]][j],metric_weights,metric_means,metric_covariances,prob_at_x)    
                kmedian_fairlet_likelihood.append(logL(k,G,dist,G_metric=True))            

        decomp_likelihood = np.sum(kmedian_fairlet_likelihood)               


        # Algorithm 1 on kmedian decomposition
        fairlet_sizes = [len(kmedian_fairlets[i]) for i in range(len(kmedian_fairlet_centers))] # size of fairlets
        Dbar=[]
        for i in range(len(kmedian_fairlet_centers)):
            for j in range(fairlet_sizes[i]):
                Dbar.append(data[kmedian_fairlet_centers[i]])
        Dbar = np.asarray(Dbar)
         
        # Computing mixture model on decomposition
        gm = GaussianMixture(n_components=components, random_state=random_state,covariance_type=cov_type,max_iter=100,n_init=3,init_params='k-means++').fit(Dbar)
        gamma = gm.predict_proba(Dbar)
        weights = gm.weights_
        means = gm.means_
        covariances = gm.covariances_

        Dbar_cost = MHD_GMM_Cost(gamma,Dbar,weights,means,covariances) 
        total_cost = decomp_cost + Dbar_cost

        print('Kmedian Total cost:',total_cost)        
        
        # Assigning center responsibilities to fairlet members with correct index
        center_gamma = gm.predict_proba(data[kmedian_fairlet_centers]) # responsibility of centers
        gamma=np.zeros([data.shape[0],components])
        for i in range(len(kmedian_fairlet_centers)):
                gamma[kmedian_fairlets[i]] = center_gamma[i]
    
        # Fairness
        H1,H_ratio1,balance1,p1 = soft_fairness(gamma,A)   # for category 1 
        H2,H_ratio2,balance2,p2 = soft_fairness(gamma,1-A) # for category 2
        balance = np.min([balance1,balance2])
        H_ratio = np.min([H_ratio1,H_ratio2])
        
        D_kmedian.append(decomp_cost)
        C_kmedian.append(total_cost)
        B_kmedian.append(balance)
        H_kmedian.append(H_ratio)
        L_kmedian.append(decomp_likelihood)        
    
    # Generating mean and standard deviation values for the current number of mixture components
    cost_mean_full    = np.mean(C_full)
    cost_std_full     = np.std(C_full)
    balance_mean_full = np.mean(B_full)
    balance_std_full  = np.std(B_full)
    entropy_mean_full = np.mean(H_full)
    entropy_std_full  = np.std(H_full)


    decomp_mean_mcf    = np.mean(D_mcf)
    decomp_std_mcf     = np.std(D_mcf)
    cost_mean_mcf    = np.mean(C_mcf)
    cost_std_mcf     = np.std(C_mcf)
    balance_mean_mcf = np.mean(B_mcf)
    balance_std_mcf  = np.std(B_mcf)
    entropy_mean_mcf = np.mean(H_mcf)
    entropy_std_mcf  = np.std(H_mcf)
    like_mean_mcf = np.mean(L_mcf)
    like_std_mcf  = np.std(L_mcf)
    
    decomp_mean_kmedian    = np.mean(D_kmedian)
    decomp_std_kmedian     = np.std(D_kmedian)    
    cost_mean_kmedian    = np.mean(C_kmedian)
    cost_std_kmedian     = np.std(C_kmedian)
    balance_mean_kmedian = np.mean(B_kmedian)
    balance_std_kmedian  = np.std(B_kmedian)
    entropy_mean_kmedian = np.mean(H_kmedian)
    entropy_std_kmedian  = np.std(H_kmedian)
    like_mean_kmedian = np.mean(L_kmedian)
    like_std_kmedian  = np.std(L_kmedian)
    
    # Saving mean and standard deviation values for the current number of components
    Cost_full[components-2,0]        = cost_mean_full
    Cost_mcf[components-2,0]         = cost_mean_mcf
    Cost_kmedian[components-2,0]     = cost_mean_kmedian
    
    Decomp_mcf[components-2,0]         = decomp_mean_mcf
    Decomp_kmedian[components-2,0]     = decomp_mean_kmedian

    Balance_full[components-2,0]     = balance_mean_full 
    Balance_mcf[components-2,0]      = balance_mean_mcf
    Balance_kmedian[components-2,0]  = balance_mean_kmedian

    Entropy_full[components-2,0]     = entropy_mean_full 
    Entropy_mcf[components-2,0]      = entropy_mean_mcf
    Entropy_kmedian[components-2,0]  = entropy_mean_kmedian

    Cost_full[components-2,1]        = cost_std_full
    Cost_mcf[components-2,1]         = cost_std_mcf
    Cost_kmedian[components-2,1]     = cost_std_kmedian
    
    Decomp_mcf[components-2,1]         = decomp_std_mcf
    Decomp_kmedian[components-2,1]     = decomp_std_kmedian

    Balance_full[components-2,1]     = balance_std_full 
    Balance_mcf[components-2,1]      = balance_std_mcf
    Balance_kmedian[components-2,1]  = balance_std_kmedian

    Entropy_full[components-2,1]     = entropy_std_full 
    Entropy_mcf[components-2,1]      = entropy_std_mcf
    Entropy_kmedian[components-2,1]  = entropy_std_kmedian
    
    
    Like_mcf[components-2,1]      = like_std_mcf
    Like_kmedian[components-2,1]  = like_std_kmedian
    Like_mcf[components-2,0]      = like_mean_mcf
    Like_kmedian[components-2,0]  = like_mean_kmedian
    
print('\nDone!')

# Saving results

In [ ]:
import pickle
output = open('results/census/Cost_full.pkl', 'wb')
pickle.dump(Cost_full, output)
output.close()
output = open('results/census/Cost_vanilla.pkl', 'wb')
pickle.dump(Cost_vanilla, output)
output.close()
output = open('results/census/Cost_mcf.pkl', 'wb')
pickle.dump(Cost_mcf, output)
output.close()
output = open('results/census/Cost_kmedian.pkl', 'wb')
pickle.dump(Cost_kmedian, output)
output.close()


output = open('results/census/Decomp_mcf.pkl', 'wb')
pickle.dump(Decomp_mcf, output)
output.close()
output = open('results/census/Decomp_kmedian.pkl', 'wb')
pickle.dump(Decomp_kmedian, output)
output.close()

output = open('results/census/Balance_full.pkl', 'wb')
pickle.dump(Balance_full, output)
output.close()
output = open('results/census/Balance_mcf.pkl', 'wb')
pickle.dump(Balance_mcf, output)
output.close()
output = open('results/census/Balance_kmedian.pkl', 'wb')
pickle.dump(Balance_kmedian, output)
output.close()

output = open('results/census/Entropy_full.pkl', 'wb')
pickle.dump(Entropy_full, output)
output.close()
output = open('results/census/Entropy_mcf.pkl', 'wb')
pickle.dump(Entropy_mcf, output)
output.close()
output = open('results/census/Entropy_kmedian.pkl', 'wb')
pickle.dump(Entropy_kmedian, output)
output.close()

output = open('results/census/Like_mcf.pkl', 'wb')
pickle.dump(Like_mcf, output)
output.close()
output = open('results/census/Like_kmedian.pkl', 'wb')
pickle.dump(Like_kmedian, output)
output.close()